In [2]:
#데이터베이스 활용 위해 pymysql 사용
import pymysql
connection = pymysql.connect(host="heena-db.cj3vpmozuotg.ap-northeast-2.rds.amazonaws.com",
port=3306,
user="root",
password="12569432!!!**",
database="subproject1",
charset="utf8mb4")

In [20]:
#크롤링 한 파일 불러오기
import pandas as pd
restaurant_df = pd.read_csv('restaurant_crawling.csv', encoding='cp949')
restaurant_df = restaurant_df.fillna(0) #NAN값 0으로 만들어주기 #그렇지 않으면 에러 뜸
restaurant_df.head(5)

,가게이름,카테고리,위치,평점,리뷰수
0,원조두꺼비집불오징어,해물 | 생선,서울 은평구 연서로28길 5 1 2층,4.3,206
1,구석집,호프 | 요리주점,서울 은평구 통일로83길 17-5 1층,3.7,157
2,백년가게 만포면옥 본점,한식,서울 은평구 연서로 171 1 2층,3.3,98
3,골목집갈비살,갈비,서울 은평구 연서로27길 12-9 1층,4.2,51
4,카페뤽상부르,커피전문점,서울 은평구 연서로25길 6-11 1 2층,3.8,100


In [25]:
#데이터베이스 테이블에 값 한번에 넣어주기
with connection.cursor() as cursor:
    sql = "INSERT INTO `Restaurants`(`Name`, `Category`, `Address`, `Rating`, `Review_num`) VALUES(%s, %s, %s, %s, %s)"
    mylist=[]
    
    for index, row in restaurant_df.iterrows(): #csv파일 내용 한 줄씩 가져옴
        mylist.append(tuple((row[0], row[1], row[2], float(row[3]), int(row[4]))))
    
    cursor.executemany(sql, mylist)
    
    connection.commit() 

In [82]:
#평점이 4.9 이상인 것만 추출
#distinct 사용하여 중복값 제거
with connection.cursor() as cursor:
    sql = "SELECT DISTINCT * FROM `Restaurants` WHERE `Rating` >= %s"
    cursor.execute(sql, (float(4.9)))
    result = cursor.fetchall()
    print(result)

(('미각', '샤브샤브', '서울 은평구 연서로29길 10-1 1층', 4.9, 18), ('유토피아룸카페', '카페', '서울 은평구 연서로27길 16-8 2층', 5.0, 0), ('낭풍', '찌개 | 전골', '서울 은평구 연서로29길 11-2', 5.0, 11), ('무지개양꼬치 연신내점', '양꼬치', '서울 은평구 통일로83길 13 1층', 5.0, 14), ('로마네꽁띠', '와인바', '서울 종로구 북촌로1길 30-20', 5.0, 3), ('공공재커피클럽', '카페', '서울 종로구 윤보선길 35 1층 102호', 5.0, 46), ('행복한베이커리&카페 안국점', '제과 | 베이커리', '서울 종로구 윤보선길 17', 5.0, 0), ('보뜨', '테마카페', '서울 종로구 윤보선길 28 1층 2호', 5.0, 25), ('르콘토엘', '제과 | 베이커리', '서울 종로구 윤보선길 78 1층', 5.0, 16), ('아반떼', '술집', '서울 종로구 율곡로 55', 5.0, 0), ('조스키친', '음식점', '서울 종로구 율곡로3길 74-6', 5.0, 0), ('일월카츠', '돈까스 | 우동', '서울 종로구 계동길 17', 4.9, 116), ('냉장고', '육류 | 고기', '서울 마포구 와우산로 23 규민빌딩 1층 102호', 4.9, 59), ('철인7호치킨 홍대점', '치킨', '서울 마포구 양화로6길 99-3 반지하', 5.0, 64))


In [83]:
#카페만 추출
#distinct 사용하여 중복값 제거
with connection.cursor() as cursor:
    sql = "SELECT DISTINCT * FROM `Restaurants` WHERE `Category` = %s"
    cursor.execute(sql, ('카페'))
    result = cursor.fetchall()
    result = result[:5] #일단 5개만
    print(result)

(('플럽커피', '카페', '서울 은평구 연서로29길 7-11 3층', 3.4, 139), ('탐나도다', '카페', '서울 은평구 통일로83길 5-14', 4.5, 12), ('코피티암', '카페', '서울 은평구 연서로27길 7-1 1 2층', 3.4, 32), ('제이츠커피브루어스', '카페', '서울 은평구 연서로25길 20-3 1층', 4.1, 30), ('유토피아룸카페', '카페', '서울 은평구 연서로27길 16-8 2층', 5.0, 0))


In [84]:
#종로구만 추출
#distinct 사용하여 중복값 제거
with connection.cursor() as cursor:
    sql = "SELECT DISTINCT * FROM `Restaurants` WHERE `Address` LIKE '%종로구%'"
    cursor.execute(sql)
    result = cursor.fetchall()
    result = result[:5] #일단 5개만
    print(result)

(('블루보틀 삼청 카페', '카페', '서울 종로구 북촌로5길 76 1~3층', 3.3, 931), ('황생가칼국수', '국수', '서울 종로구 북촌로5길 78', 3.6, 624), ('토속촌삼계탕', '삼계탕', '서울 종로구 자하문로5길 5', 3.7, 65), ('아키비스트', '카페', '서울 종로구 효자로13길 52 1층', 3.7, 702), ('스미스가 좋아하는 한옥', '이탈리안', '서울 종로구 삼청로 22-7', 3.0, 223))


In [85]:
#평점이 4.0 이상이고 리뷰수가 500개 이상인 것만 추출
#distinct 사용하여 중복값 제거
with connection.cursor() as cursor:
    sql = "SELECT DISTINCT * FROM `Restaurants` WHERE `Rating` >= 4.0 AND `Review_num` >= 500"
    cursor.execute(sql)
    result = cursor.fetchall()
    result = result[:5] #일단 5개만
    print(result)

(('안녕인사동', '테마카페', '서울 종로구 인사동길 49', 4.1, 511), ('오레노라멘 인사점', '일본식라면', '서울 종로구 율곡로 49-4', 4.1, 537), ('다운타우너 안국', '햄버거', '서울 종로구 북촌로 6-4 1층', 4.1, 1081), ('포비 광화문점', '카페', '서울 종로구 종로3길 17 광화문D타워 1층 21호', 4.2, 530), ('어반플랜트 합정', '카페', '서울 마포구 독막로4길 3 1~3층', 4.2, 690))
